# 02 – Preprocessing

Pipeline czyszczenia tekstu dla opinii Allegro: import preprocessora, test na przykładach, zastosowanie na train/val, porównanie przed/po.

## 1. Importy i ładowanie danych

In [2]:
import sys
from pathlib import Path

# Dodaj katalog główny projektu do ścieżki (żeby działał import src)
root = Path.cwd()
if not (root / "src").is_dir():
    root = root.parent
sys.path.insert(0, str(root))

import pandas as pd
from datasets import load_dataset
from src.preprocessing import PolishTextPreprocessor

dataset = load_dataset("allegro/klej-allegro-reviews")
train_df = dataset["train"].to_pandas()
val_df = dataset["validation"].to_pandas()
print("Train:", train_df.shape, "| Val:", val_df.shape)

Train: (9577, 2) | Val: (1002, 2)


## 2. Inicjalizacja preprocessora i test na przykładach

In [3]:
preprocessor = PolishTextPreprocessor(
    lowercase=False,
    remove_emoji=True,
    remove_stopwords=False,
)

# Test na kilku zdaniach (w tym z URL / emoji)
examples = [
    train_df["text"].iloc[0],
    "Super produkt!!! https://allegro.pl/item  Polecam 😊",
    "   Dużo   spacji   i   znaków.   ",
]
for ex in examples:
    out = preprocessor.preprocess_pipeline(ex)
    print("IN:", repr(ex[:80]) + ("..." if len(ex) > 80 else ""))
    print("OUT:", repr(out[:80]) + ("..." if len(out) > 80 else ""))
    print("---")

IN: 'Jako do ceny dobra. Przyssawka mogłaby być lepsza. Po 2 miesiącach użytkowania m'...
OUT: 'Jako do ceny dobra. Przyssawka mogłaby być lepsza. Po 2 miesiącach użytkowania m'...
---
IN: 'Super produkt!!! https://allegro.pl/item  Polecam 😊'
OUT: 'Super produkt!!! Polecam'
---
IN: '   Dużo   spacji   i   znaków.   '
OUT: 'Dużo spacji i znaków.'
---


## 3. Zastosowanie preprocessora na train i validation

In [4]:
train_df["text_clean"] = preprocessor.preprocess_series(train_df["text"])
val_df["text_clean"] = preprocessor.preprocess_series(val_df["text"])
print("Gotowe. Przykład:")
train_df[["text", "text_clean", "rating"]].head(2)

Gotowe. Przykład:


,text,text_clean,rating
0,Jako do ceny dobra. Przyssawka mogłaby być lep...,Jako do ceny dobra. Przyssawka mogłaby być lep...,3.0
1,Na słuchawkę czekałam spory czas a po zadzwoni...,Na słuchawkę czekałam spory czas a po zadzwoni...,1.0


## 4. Porównanie przed / po (długość, puste)

In [5]:
train_df["words_before"] = train_df["text"].str.split().str.len()
train_df["words_after"] = train_df["text_clean"].str.split().str.len()

print("Train – liczba słów:")
print("  przed:", train_df["words_before"].describe().round(1).to_string())
print("  po:", train_df["words_after"].describe().round(1).to_string())

empty_before = (train_df["words_before"] == 0).sum()
empty_after = (train_df["words_after"] == 0).sum()
print(f"\nPuste (0 słów): przed={empty_before}, po={empty_after}")

Train – liczba słów:
  przed: count    9577.0
mean       72.6
std        47.0
min         0.0
25%        55.0
50%        64.0
75%        83.0
max      1667.0
  po: count    9577.0
mean       72.6
std        47.0
min         0.0
25%        55.0
50%        64.0
75%        83.0
max      1667.0

Puste (0 słów): przed=397, po=397


In [6]:
pd.set_option("display.max_colwidth", 60)
sample = train_df.sample(3, random_state=42)[["text", "text_clean", "rating"]]
sample

,text,text_clean,rating
7355,Taki zestaw to 2xUSB 3.0 z tyłu obudowy i 2xUSB 3.0 z pr...,Taki zestaw to 2xUSB 3.0 z tyłu obudowy i 2xUSB 3.0 z pr...,5.0
5814,Produkt jest hmmm średni. To nie jest jakiś badziew co t...,Produkt jest hmmm średni. To nie jest jakiś badziew co t...,3.0
1383,"Być może miałem pecha, ale obydwa zakupione u mnie pierś...","Być może miałem pecha, ale obydwa zakupione u mnie pierś...",3.0


## 5. Opcjonalnie: zapis do data/processed

In [10]:
# Zapis tylko kolumn potrzebnych do dalszych faz (embeddingi, trening)
out_dir = "/Users/jakub/sentiment-classifier-pl/data/processed"
train_df[["text_clean", "rating"]].to_csv(f"{out_dir}/train_clean.csv", index=False)
val_df[["text_clean", "rating"]].to_csv(f"{out_dir}/val_clean.csv", index=False)
print(f"Zapisano {out_dir}/train_clean.csv i {out_dir}/val_clean.csv")

Zapisano /Users/jakub/sentiment-classifier-pl/data/processed/train_clean.csv i /Users/jakub/sentiment-classifier-pl/data/processed/val_clean.csv
